In [ ]:
from huggingface_hub import scan_cache_dir

# 1. Scan the entire current cache
cache_info = scan_cache_dir()

# 2. Create a deletion strategy that includes ALL revisions of ALL repositories
delete_strategy = cache_info.delete_revisions(*[
    revision.commit_hash 
    for repo in cache_info.repos 
    for revision in repo.revisions
])

# 3. Show how much space will be freed before executing
print(f"Cleaning cache...")
print(f"Total space to be freed: {delete_strategy.expected_freed_size_str}")

# 4. Execute the cleanup
delete_strategy.execute()

print("Model cache completely cleaned!")

Limpiando caché...
Espacio total que se liberará: 18.5G
¡Caché de modelos completamente limpia!


In [ ]:
from huggingface_hub import HfApi

api = HfApi()

# Fetch trending models with expanded metadata
trending = api.list_models(
    sort="likes7d", 
    limit=50,
    expand=["safetensors", "cardData"]
)

# Containers for our tables
all_models = []
filtered_models = []

for model in trending:
    m_id = model.id
    tags = [t.lower() for t in (model.tags or [])]
    
    # --- Classification Logic ---
    is_lora = "lora" in tags or "peft" in tags or "adapter" in m_id.lower() or "lora" in m_id.lower()
    is_gguf = "gguf" in tags or m_id.lower().endswith(".gguf") or "gguf" in m_id.lower()
    
    quant_tags = {"awq", "gptq", "exl2", "bitsandbytes", "quantized"}
    is_quant = any(q in tags for q in quant_tags) or "quant" in m_id.lower()
    
    base_model_pointer = getattr(model.cardData, 'base_model', None)
    
    if is_lora:
        category = "LoRA"
    elif is_gguf:
        category = "GGUF"
    elif is_quant:
        category = "Quant"
    elif base_model_pointer:
        category = "Fine-tuned"
    else:
        category = "Base"

    # --- Parameter Extraction ---
    raw_params = 0
    params_display = "N/A"
    if hasattr(model, 'safetensors') and model.safetensors:
        raw_params = model.safetensors.get('total', 0)
        if raw_params:
            params_display = f"{raw_params / 1e9:.1f}B" if raw_params >= 1e9 else f"{raw_params / 1e6:.1f}M"

    # Store for Table 1
    all_models.append((m_id, category, params_display))
    
    # Store for Table 2 (Base/Fine-tuned only AND < 10B)
    # We only include if raw_params > 0 to avoid empty metadata entries
    if category in ["Base", "Fine-tuned"] and 0 < raw_params < 10e9:
        filtered_models.append((m_id, category, params_display))

# --- Printing Table 1 ---
print("TABLE 1: TOP 50 TRENDING (ALL TYPES)")
print(f"{'Model ID':<55} | {'Type':<12} | {'Params'}")
print("-" * 80)
for m in all_models:
    print(f"{m[0]:<55} | {m[1]:<12} | {m[2]}")

print("\n" + "="*80 + "\n")

# --- Printing Table 2 ---
print("TABLE 2: BASE & FINE-TUNED MODELS (< 10B PARAMS)")
print(f"{'Model ID':<55} | {'Type':<12} | {'Params'}")
print("-" * 80)
if not filtered_models:
    print("No models found matching these specific criteria in the current trend list.")
for m in filtered_models:
    print(f"{m[0]:<55} | {m[1]:<12} | {m[2]}")

c:\Users\emman\Desktop\PROYECTOS_VS_CODE\GITHUB\Model-Quantization\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TABLE 1: TOP 50 TRENDING (ALL TYPES)
Model ID                                                | Type         | Params
--------------------------------------------------------------------------------
zai-org/GLM-5                                           | Base         | 753.9B
openbmb/MiniCPM-o-4_5                                   | Base         | 9.4B
openbmb/MiniCPM-SALA                                    | Base         | 9.5B
Qwen/Qwen3-Coder-Next                                   | Base         | 79.7B
moonshotai/Kimi-K2.5                                    | Base         | 170.7B
zai-org/GLM-OCR                                         | Base         | N/A
mistralai/Voxtral-Mini-4B-Realtime-2602                 | Fine-tuned   | N/A
Nanbeige/Nanbeige4.1-3B                                 | Fine-tuned   | 3.9B
ACE-Step/Ace-Step1.5                                    | Base         | N/A
circlestone-labs/Anima                                  | Base         | N/A
stepfun-ai/Step-3.5-F

In [ ]:
import re
import torch
from transformers import AutoConfig, AutoModelForCausalLM, AutoModel

SKIP_MODEL_TYPES = {
    "qwen3_vl",
    "paddleocr_vl",
    "vibevoice",      # unsupported in transformers right now
}

SKIP_KEYWORDS = ("vl", "vision", "multimodal", "asr", "ocr", "speech")

def extract_missing_package(err: Exception):
    msg = str(err)
    m = re.search(r"packages that were not found.*?:\s*([a-zA-Z0-9_\-]+)", msg)
    if m:
        return m.group(1)
    m = re.search(r"No module named '([^']+)'", msg)
    if m:
        return m.group(1)
    return None

print("\n" + "="*80)
print("LOADING TABLE 2 MODELS ON META DEVICE")
print("="*80)

loaded_meta_models = {}

for model_id, category, params in filtered_models:
    try:
        config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
        model_type = (getattr(config, "model_type", "") or "").lower()

        # Skip known non-text or unsupported types
        if model_type in SKIP_MODEL_TYPES:
            print(f"[SKIP] {model_id:<55} | {category:<10} | {params} | model_type={model_type}")
            continue

        # Optional heuristic skip (helps avoid OCR/ASR/VL even if model_type is generic)
        if any(k in model_id.lower() for k in SKIP_KEYWORDS):
            print(f"[SKIP] {model_id:<55} | {category:<10} | {params} | heuristic_skip")
            continue

        with torch.device("meta"):
            try:
                model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)
            except Exception:
                model = AutoModel.from_config(config, trust_remote_code=True)

        loaded_meta_models[model_id] = model
        print(f"[OK]   {model_id:<55} | {category:<10} | {params} | model_type={model_type}")

    except ImportError as e:
        missing = extract_missing_package(e)
        if missing:
            print(f"[SKIP] {model_id:<55} | {category:<10} | {params} | missing dep: {missing} (pip install {missing})")
        else:
            print(f"[FAIL] {model_id:<55} | {category:<10} | {params} | ImportError: {str(e)[:120]}")

    except ValueError as e:
        # Common case: unknown architecture
        if "does not recognize this architecture" in str(e).lower():
            print(f"[SKIP] {model_id:<55} | {category:<10} | {params} | unsupported by transformers")
        else:
            print(f"[FAIL] {model_id:<55} | {category:<10} | {params} | ValueError: {str(e)[:120]}")

    except Exception as e:
        print(f"[FAIL] {model_id:<55} | {category:<10} | {params} | {type(e).__name__}: {str(e)[:120]}")



LOADING TABLE 2 MODELS ON META DEVICE


Encountered exception while importing minicpmo: No module named 'minicpmo'
Encountered exception while importing minicpmo: No module named 'minicpmo'


[SKIP] openbmb/MiniCPM-o-4_5                                   | Base       | 9.4B | missing dep: minicpmo (pip install minicpmo)


Encountered exception while importing fla: No module named 'fla'
Encountered exception while importing fla: No module named 'fla'


[SKIP] openbmb/MiniCPM-SALA                                    | Base       | 9.5B | missing dep: fla (pip install fla)
[OK]   Nanbeige/Nanbeige4.1-3B                                 | Fine-tuned | 3.9B | model_type=llama
[OK]   inference-net/Schematron-3B                             | Fine-tuned | 3.2B | model_type=llama
[OK]   UCSB-SURFI/VulnLLM-R-7B                                 | Fine-tuned | 7.6B | model_type=qwen2


The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[OK]   ytu-ce-cosmos/Turkish-Gemma-9b-T1                       | Fine-tuned | 9.2B | model_type=gemma2
[SKIP] Qwen/Qwen3-ASR-1.7B                                     | Base       | 2.3B | unsupported by transformers
[SKIP] kugelaudio/kugelaudio-0-open                            | Base       | 9.3B | unsupported by transformers
[OK]   FutureMa/Eva-4B-V2                                      | Fine-tuned | 4.0B | model_type=qwen3
[OK]   Octen/Octen-Embedding-8B                                | Fine-tuned | 7.6B | model_type=qwen3
[FAIL] meta-llama/Llama-3.1-8B-Instruct                        | Fine-tuned | 8.0B | OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.1-8B-
[SKIP] Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice                    | Base       | 1.9B | unsupported by transformers
[FAIL] deepseek-ai/DeepSeek-OCR-2                              | Base       | 3.4B | ImportError: cannot import name 'LlamaFlashAttention2'

c:\Users\emman\Desktop\PROYECTOS_VS_CODE\GITHUB\Model-Quantization\venv\Lib\site-packages\transformers\modeling_rope_utils.py:936: FutureWarning: `rope_config_validation` is deprecated and has been removed. Its functionality has been moved to RotaryEmbeddingConfigMixin.validate_rope method. PreTrainedConfig inherits this class, so please call self.validate_rope() instead. Also, make sure to use the new rope_parameters syntax. You can call self.standardize_rope_params() in the meantime.
  warnings.warn(
Unrecognized keys in `rope_parameters` for 'rope_type'='default': {'mrope_section'}


[SKIP] PaddlePaddle/PaddleOCR-VL-1.5                           | Fine-tuned | 958.6M | model_type=paddleocr_vl
[SKIP] microsoft/VibeVoice-ASR                                 | Base       | 8.7B | unsupported by transformers
[SKIP] trillionlabs/gWorld-8B                                  | Base       | 8.8B | model_type=qwen3_vl
